# Line most tangent to three Mohr circles

This notebook develops the math for drawing the line which is most tangent to three or more different Mohr circles (as is needed for derivation of drained parameters from a CU test with three subsamples.

In [ ]:
import numpy as np
import os
from copy import deepcopy
import pandas as pd
pd.options.display.max_columns = 200
pd.options.display.max_rows = 1000

In [ ]:
from plotly import tools, subplots
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
import plotly.figure_factory as ff
from plotly.colors import DEFAULT_PLOTLY_COLORS
from plotly.offline import init_notebook_mode
init_notebook_mode()
pio.templates.default = 'plotly_white'

In [ ]:
sigma3s = np.array([100, 200, 400])
sigma1s = np.array([400, 650, 1000])

In [ ]:
radii = 0.5 * (sigma1s - sigma3s)
centers = 0.5 * (sigma1s + sigma3s)
thetas = np.linspace(0, np.pi, 250)

In [ ]:
fig = subplots.make_subplots(rows=1, cols=1, print_grid=False)
for i, _r in enumerate(radii):
    _data = go.Scatter(
        x=centers[i] + _r * np.cos(thetas),
        y=_r * np.sin(thetas), showlegend=False, mode='lines',name='Legend name')
    fig.append_trace(_data, 1, 1)
fig['layout']['xaxis1'].update(title=r'$ \sigma \ \text{[kPa]}$', range=(0, 1000), dtick=100)
fig['layout']['yaxis1'].update(title=r'$ \tau \ \text{[kPa]}$', scaleanchor='x', scaleratio=1.0, range=(0, 450))
fig['layout'].update(height=500, width=900)
fig.show()

The failure criterion can be written as:

$$ \tau = c^{\prime} + \sigma^{\prime} \tan \varphi^{\prime} $$

or in $ ax +by + c=0$ form:

$$ \tan \varphi^{\prime} \sigma^{\prime} - \tau + c^{\prime} = 0 $$

where $ a = \tan \varphi^{\prime} $, $ b = -1 $ and $ c = c^{\prime} $.

The aim is to minimise the distance from the center of each Mohr circle to the failure criterion for each of the three circles. The distance from a point ($x_0, y_0$) to a line can be expressed as:

$$ d = \frac{\left| a x_0 + b y_0 + c\right|}{\sqrt{a^2 + b^2}} $$ 

For our problem, this can be written as:

$$ d = \frac{\left| \tan \varphi^{\prime} \frac{\sigma_1^{\prime} + \sigma_3^{\prime}}{2} + c^{\prime} \right|}{\sqrt{\tan^2 \varphi^{\prime} + 1}} $$ 

This distance needs to be as close to the radius of each circle as possible. The difference can be expressed by subtracting this radius ($ \frac{\sigma_1^{\prime} - \sigma_3^{\prime}}{2} $)

In [ ]:
from scipy.optimize import minimize

In [ ]:
def distance_func(x, sigma1_effs, sigma3_effs):
    """
    Defines the distance function, x[0] = tan phi and x[1] = c'
    Returns the sum of all distances
    """
    ds = np.zeros(sigma1_effs.__len__())
    for i, (_sigma1, _sigma3) in enumerate(zip(sigma1_effs, sigma3_effs)):
        ds[i] = (np.abs(x[0] * 0.5 * (_sigma1 + _sigma3) + x[1]) / \
            np.sqrt(x[0] ** 2 + 1) - (0.5 * (_sigma1 - _sigma3))) ** 2
        
    return ds.sum()

In [ ]:
result = distance_func(np.array([np.tan(np.radians(25)), 5]), sigma1s, sigma3s)
result.sum()

In [ ]:
np.tan(np.radians(15)), np.tan(np.radians(40))

In [ ]:
minimised_params = minimize(
    distance_func, # A function of one of more variables
    x0=np.array([np.tan(np.radians(25)), 50]), # Initial guess for the values which minimise the function
    method='SLSQP', # Method used for the minimisation
    args=(sigma1s, sigma3s), # Function arguments, two arguments in this case
    bounds=((np.tan(np.radians(15)), np.tan(np.radians(40))), (0, None)) # Bounds between which the parameters may vary (tuple of (min, max) values)
)

minimised_params

In [ ]:
np.rad2deg(np.arctan(minimised_params.x[0]))

In [ ]:
effective_friction_angle = round(np.rad2deg(np.arctan(minimised_params.x[0])), 1)
effective_cohesion = round(minimised_params.x[1], 1)
print(effective_friction_angle, effective_cohesion)

In [ ]:
fig = subplots.make_subplots(rows=1, cols=1, print_grid=False)
for i, _r in enumerate(radii):
    _data = go.Scatter(
        x=centers[i] + _r * np.cos(thetas),
        y=_r * np.sin(thetas), showlegend=False, mode='lines',name='Legend name')
    fig.append_trace(_data, 1, 1)
_data = go.Scatter(
    x=np.linspace(0, 1000, 250),
    y=minimised_params.x[1] + np.linspace(0, 1000, 250) * minimised_params.x[0],
    showlegend=False, mode='lines',name='Legend name', line=dict(color='black', dash='dot'))
fig.append_trace(_data, 1, 1)
fig['layout']['xaxis1'].update(title=r'$ \sigma \ \text{[kPa]}$', dtick=100)
fig['layout']['yaxis1'].update(title=r'$ \tau \ \text{[kPa]}$', scaleanchor='x', scaleratio=1.0)
fig['layout'].update(height=500, width=900)
fig.show()